In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F

In [2]:
data = pd.read_csv('data/processed_data.csv')

In [3]:
viewport = data.copy()
viewport[(viewport.season == 2021) & (viewport['round'] == 3)][:22]

,season,round,country,url,driver,age,grid,podium,driver_points,driver_wins,...,constructor_minardi,constructor_racing_point,constructor_red_bull,constructor_renault,constructor_sauber,constructor_super_aguri,constructor_toro_rosso,constructor_toyota,constructor_virgin,constructor_williams
7188,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,bottas,31,1,3,32.0,0.0,...,False,False,False,False,False,False,False,False,False,False
7189,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,hamilton,36,2,1,69.0,2.0,...,False,False,False,False,False,False,False,False,False,False
7190,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,max_verstappen,23,3,2,61.0,1.0,...,False,False,True,False,False,False,False,False,False,False
7191,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,perez,31,4,4,22.0,0.0,...,False,False,True,False,False,False,False,False,False,False
7192,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,sainz,26,5,11,14.0,0.0,...,False,False,False,False,False,False,False,False,False,False
7193,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,ocon,24,6,7,8.0,0.0,...,False,False,False,False,False,False,False,False,False,False
7194,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,norris,21,7,5,37.0,0.0,...,False,False,False,False,False,False,False,False,False,False
7195,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,leclerc,23,8,6,28.0,0.0,...,False,False,False,False,False,False,False,False,False,False
7196,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,gasly,25,9,10,7.0,0.0,...,False,False,False,False,False,False,False,False,False,False
7197,2021,3,Portugal,http://en.wikipedia.org/wiki/2021_Portuguese_G...,vettel,33,10,13,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False


In [4]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season<2021]
train.head()

,season,round,country,url,driver,age,grid,podium,driver_points,driver_wins,...,constructor_minardi,constructor_racing_point,constructor_red_bull,constructor_renault,constructor_sauber,constructor_super_aguri,constructor_toro_rosso,constructor_toyota,constructor_virgin,constructor_williams
0,2003,1,Australia,http://en.wikipedia.org/wiki/2003_Australian_G...,michael_schumacher,34,1,0,5.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,2003,1,Australia,http://en.wikipedia.org/wiki/2003_Australian_G...,barrichello,30,2,0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,2003,1,Australia,http://en.wikipedia.org/wiki/2003_Australian_G...,montoya,27,3,0,8.0,0.0,...,False,False,False,False,False,False,False,False,False,True
3,2003,1,Australia,http://en.wikipedia.org/wiki/2003_Australian_G...,frentzen,35,4,0,3.0,0.0,...,False,False,False,False,True,False,False,False,False,False
4,2003,1,Australia,http://en.wikipedia.org/wiki/2003_Australian_G...,panis,36,5,0,0.0,0.0,...,False,False,False,False,False,False,False,True,False,False


In [5]:
X_train = train.drop(['driver', 'country', 'podium', 'url'], axis = 1)
y_train = train.podium

scaler = StandardScaler()
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: podium, dtype: int64

In [6]:
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [7]:
class F1RacePrediction(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential (
        nn.Flatten(),
        nn.Linear(96, 75),
        nn.ReLU(),
        nn.Linear(75, 25),
        nn.ReLU(),
        nn.Linear(25,50),
        nn.ReLU(),
        nn.Linear(50,10)
    )

  def forward(self, x):
    return self.layers(x)

In [8]:
X_train = torch.FloatTensor(X_train.to_numpy())
y_train = torch.LongTensor(y_train)

In [9]:
torch.manual_seed(42)
model = F1RacePrediction()
model

F1RacePrediction(
  (layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=96, out_features=75, bias=True)
    (2): ReLU()
    (3): Linear(in_features=75, out_features=25, bias=True)
    (4): ReLU()
    (5): Linear(in_features=25, out_features=50, bias=True)
    (6): ReLU()
    (7): Linear(in_features=50, out_features=10, bias=True)
  )
)

In [10]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), 
                            lr=1e-4)

In [11]:
# Train our model!
# Epochs? (one run thru all the training data in our network)
epochs = 1700
losses = []
for i in range(epochs):
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = loss_function(y_pred, y_train) # predicted values vs the y_train
    loss.backward()
    optimizer.step()
    
    if i % 10 == 0:
        print(f'Epoch: {i} and loss: {loss}')

Epoch: 0 and loss: 2.254661798477173
Epoch: 10 and loss: 2.235753059387207
Epoch: 20 and loss: 2.2164759635925293
Epoch: 30 and loss: 2.1963794231414795
Epoch: 40 and loss: 2.174882650375366
Epoch: 50 and loss: 2.1513237953186035
Epoch: 60 and loss: 2.124936819076538
Epoch: 70 and loss: 2.0948047637939453
Epoch: 80 and loss: 2.0598785877227783
Epoch: 90 and loss: 2.0189380645751953
Epoch: 100 and loss: 1.970524787902832
Epoch: 110 and loss: 1.9130679368972778
Epoch: 120 and loss: 1.8447110652923584
Epoch: 130 and loss: 1.7636560201644897
Epoch: 140 and loss: 1.6684401035308838
Epoch: 150 and loss: 1.5587533712387085
Epoch: 160 and loss: 1.435711145401001
Epoch: 170 and loss: 1.3017058372497559
Epoch: 180 and loss: 1.1603689193725586
Epoch: 190 and loss: 1.0165573358535767
Epoch: 200 and loss: 0.8760799765586853
Epoch: 210 and loss: 0.7451363801956177
Epoch: 220 and loss: 0.6291887760162354
Epoch: 230 and loss: 0.5317816138267517
Epoch: 240 and loss: 0.4536808729171753
Epoch: 250 and lo

In [12]:
from copy import deepcopy

torch.save(deepcopy(model.state_dict()), 'racemodel.pth')

In [13]:
def scorecard_ts(season, model2):
    df = data.copy()
    df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)
    score = 0
    count = 0
    predicted = 0
    for circuit in df[df.season == season]['round'].unique():
        count += 1
        winner = data[(data.season == season) & (data['round'] == circuit) & (data['podium'] == 1)].grid
        try:
            winner = winner.to_numpy()[0]
        except:
            winner = None
        model2.eval()

        conf = 1
        guess = 0
        pred = False
        for grid in range(20):
            test = df[(df.season == season) & (df['round'] == circuit) & (df['grid'] == grid + 1)]
            X_test = test.drop(['driver', 'country', 'podium', 'url'], axis=1)
            try:
                X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
            except:
                continue
            X_test = torch.Tensor(X_test.to_numpy())
            prediction = model2(X_test)
            prob = F.softmax(prediction, dim=1)
            top_p, top_class = prob.topk(1, dim=1)
            if prediction.argmax().item():
                predicted += 1
                print(
                    f'{"CORRECTLY" if grid + 1 == winner else "INCORRECTLY"} predicted. P{grid + 1} should win in round {circuit} (actual winner was P{winner})')
                score += 1 if grid + 1 == winner else 0
                pred = True
                break
            elif top_p[0][0] < conf:
                conf = top_p
                guess = grid + 1

        if not pred:
            predicted += 1
            print(
                f'{"CORRECTLY" if guess == winner else "INCORRECTLY"} predicted. P{guess} should win in round {circuit} (actual winner was P{winner})')
            score += 1 if guess == winner else 0

    print(f'{score} out of {predicted} predicted races')

In [16]:
model2 = F1RacePrediction()
model2.load_state_dict(torch.load('racemodel.pth', weights_only=True))
scorecard_ts(2023, model2)

CORRECTLY predicted. P1 should win in round 1 (actual winner was P1)
INCORRECTLY predicted. P2 should win in round 2 (actual winner was P1)
CORRECTLY predicted. P1 should win in round 3 (actual winner was P1)
INCORRECTLY predicted. P2 should win in round 4 (actual winner was P3)
CORRECTLY predicted. P9 should win in round 5 (actual winner was P9)
CORRECTLY predicted. P1 should win in round 6 (actual winner was P1)
CORRECTLY predicted. P1 should win in round 7 (actual winner was P1)
CORRECTLY predicted. P1 should win in round 8 (actual winner was P1)
CORRECTLY predicted. P1 should win in round 9 (actual winner was P1)
CORRECTLY predicted. P1 should win in round 10 (actual winner was P1)
CORRECTLY predicted. P2 should win in round 11 (actual winner was P2)
CORRECTLY predicted. P6 should win in round 12 (actual winner was P6)
CORRECTLY predicted. P1 should win in round 13 (actual winner was P1)
CORRECTLY predicted. P2 should win in round 14 (actual winner was P2)
INCORRECTLY predicted. P1